<a href="https://colab.research.google.com/github/andres9119/andres9119/blob/main/mortalidad_municipios.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Cargar los datos de cada hoja en un DataFrame
datos_2017 = pd.read_excel('mortalidad_villarosario.xlsx', sheet_name='2017')
datos_2018 = pd.read_excel('mortalidad_villarosario.xlsx', sheet_name='2018')
datos_2019 = pd.read_excel('mortalidad_villarosario.xlsx', sheet_name='2019')
datos_2020 = pd.read_excel('mortalidad_villarosario.xlsx', sheet_name='2020')
datos_2021 = pd.read_excel('mortalidad_villarosario.xlsx', sheet_name='2021')

# Solicitar al usuario que ingrese el año que desea ver
año_elegido = input("Por favor, ingresa el año que deseas ver (por ejemplo, 2018): ")

# Seleccionar el DataFrame correspondiente al año elegido
if año_elegido == '2017':
    datos_año = datos_2017
elif año_elegido == '2018':
    datos_año = datos_2018
elif año_elegido == '2019':
    datos_año = datos_2019
elif año_elegido == '2020':
    datos_año = datos_2020
elif año_elegido == '2021':
    datos_año = datos_2021
else:
    print("El año ingresado no es válido.")

datos_año.head()


In [ ]:
 (print(datos_año.columns))
 type(año_elegido)
 año_elegido
 año_elegido = int(año_elegido)

In [ ]:
# Verificar si la columna del año elegido existe en el DataFrame
if año_elegido in datos_año.columns:
    # Convertir los valores de la columna del año elegido a tipo numérico si es posible
    datos_año[año_elegido] = pd.to_numeric(datos_año[año_elegido], errors='coerce')
    # Ordenar el DataFrame por la columna del año elegido de mayor a menor
    datos_ordenados = datos_año.sort_values(by=año_elegido, ascending=False)

else:
    print("El año ingresado no es válido o no existe en el DataFrame.")

datos_ordenados.head(5)

In [ ]:
# Solicitar al usuario que ingrese la población total
poblacion_total = float(input("Ingrese la población total: "))

# Calcular las muertes total como la sumatoria de los valores en la columna del año elegido
tasa_muertes_total = datos_ordenados[año_elegido].sum()
muertes_total=(tasa_muertes_total*poblacion_total)/100000
print("el total de muertes es-->  " ,muertes_total)

In [ ]:
# Calcular el tercer cuartil
tercer_cuartil = datos_ordenados[año_elegido].quantile(0.75)

# Filtrar los datos donde el subgrupo no sea "TOTAL" ni el sexo sea "MASCULINO"
datos_filtrados = datos_ordenados[ (datos_ordenados['Subgrupo'] != 'TOTAL') &
                                   (datos_ordenados['Sexo'] != 'MASCULINO') &
                                   (datos_ordenados['Sexo'] != 'FEMENINO')]


# Mostrar los primeros N registros del cuartil más alto
datos_filtrados.head(10)



In [ ]:


# Calcular la columna de personas afectadas
datos_filtrados.loc[:, 'personas_afectadas'] = datos_filtrados[año_elegido] * poblacion_total / 100000

# Calcular el porcentaje de afectación para cada número de personas atendidas
datos_filtrados.loc[:, 'porcentaje_de_afectacion'] = (datos_filtrados['personas_afectadas'] / muertes_total) * 100

# Filtrar los datos por encima del tercer cuartil
# datos_filtrados = datos_filtrados[datos_filtrados[año_elegido] > tercer_cuartil]

# Mostrar los primeros N registros con la nueva columna
datos_filtrados.head(10)



In [ ]:
# Solicitar al usuario que ingrese el número de datos que desea trabajar
cantidad_datos = int(input("Por favor, ingrese el número de datos que desea puntuar para priorizar: "))

# Seleccionar el número de datos especificado por el usuario
datos_trabajo = datos_filtrados.head(cantidad_datos)

# Mostrar el DataFrame con el número especificado de datos
print("Los primeros", cantidad_datos, "datos para trabajar son:")
datos_trabajo

In [ ]:
# Crear la nueva columna "magnitud" en el DataFrame datos_trabajo
datos_trabajo.loc[:, 'magnitud'] = datos_trabajo['porcentaje_de_afectacion'].apply(lambda x: 10 if x >= 50 else (8 if 5 <= x < 50 else (6 if 0.5 <= x < 4.9 else 4)))

# Mostrar el DataFrame con la nueva columna
print("Datos con la nueva columna 'magnitud':")
datos_trabajo

In [ ]:
# Crear listas vacías para almacenar los valores ingresados por el usuario
severidad = []
eficacia = []
factibilidad = []

# Iterar sobre los primeros 'n' registros del DataFrame
for i in range(len(datos_trabajo)):
    # Obtener el nombre del subgrupo para mostrarlo en el mensaje
    nombre_subgrupo = datos_trabajo.iloc[i]['Subgrupo']

    # Solicitar la entrada del usuario para cada nueva columna
    valor_severidad = float(input(f"Ingrese el valor de severidad para '{nombre_subgrupo}': "))
    valor_eficacia = float(input(f"Ingrese el valor de eficacia para '{nombre_subgrupo}': "))
    valor_factibilidad = float(input(f"Ingrese el valor de factibilidad para '{nombre_subgrupo}': "))

    # Agregar los valores ingresados a las listas respectivas
    severidad.append(valor_severidad)
    eficacia.append(valor_eficacia)
    factibilidad.append(valor_factibilidad)

# Agregar las listas como nuevas columnas en el DataFrame datos_trabajo
datos_trabajo.loc[:, 'severidad'] = severidad
datos_trabajo.loc[:, 'eficacia'] = eficacia
datos_trabajo.loc[:, 'factibilidad'] = factibilidad

# Mostrar el DataFrame actualizado con las nuevas columnas
print("Datos con las nuevas columnas 'severidad', 'eficacia' y 'factibilidad':")
datos_trabajo

In [ ]:
# Calcular el valor de la nueva columna
puntuacion_hanlon = (datos_trabajo['porcentaje_de_afectacion'] + datos_trabajo['severidad']) * datos_trabajo['eficacia'] * datos_trabajo['factibilidad']

# Convertir la Serie puntuacion_hanlon en un DataFrame
df_puntuacion_hanlon = puntuacion_hanlon.to_frame()

# Agregar la nueva columna al DataFrame con el nombre "puntuación Hanlon" utilizando .loc[]
datos_trabajo.loc[:, 'puntuación Hanlon'] = puntuacion_hanlon

# Mostrar el DataFrame actualizado con la nueva columna
print("Datos con la nueva columna 'puntuación Hanlon':")
datos_trabajo

In [ ]:
# Ordenar el DataFrame por la columna 'puntuación Hanlon' de mayor a menor
df_ordenado = datos_trabajo.sort_values(by='puntuación Hanlon', ascending=False)

In [ ]:
# Seleccionar las columnas que se desean mostrar
columnas_mostrar = ['Grupo','Subgrupo', 'personas_afectadas',  'puntuación Hanlon']
pd.set_option('display.expand_frame_repr', False)
# Mostrar el DataFrame ordenado con las columnas seleccionadas
df_ordenado[columnas_mostrar]